- 以降の、それぞれのセルにカーソルを合わせて、Shift + Enter キーを押すことで、そのセルのコードを実行することができます。

- 基本的には、上のセルから順番にコードを実行していきます。

# ライブラリの読み込み

In [ ]:
from numpy import argmax
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.utils import plot_model

# データを読み込む

In [ ]:
# MNIST データ（手書き数字画像60,000枚と、テスト画像10,000枚を集めた、画像データセット）を読み込む
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 読み込んだデータの、次元ごとの要素数を表示
print("x_train（学習用の説明変数データ） の要素数: ")
print(x_train.shape)
print("y_train（学習用の目的変数データ） の要素数: ")
print(y_train.shape)
print("x_test（検証用の説明変数データ） の要素数: ")
print(x_test.shape)
print("y_test（検証用の目的変数データ） の要素数: ")
print(y_test.shape)

# 読み込んだ学習用データから 1件抜き出してその内容を表示
print("x_train のデータ 1件分のサンプル（ 28 x 28 の画像データが二次元配列の形で表されている）: ")
print(x_train[0])
print("y_train のデータ 10件分のサンプル（ 0 - 9 いずれかの正解データが格納されている）: ")
print(y_train[:10])

# MNISTデータを画像として表示
print("MNISTデータを画像として表示")
W = 8  # 横に並べる個数
H = 4   # 縦に並べる個数
fig = plt.figure(figsize=(W, H))
fig.subplots_adjust(left=0, right=1, bottom=0, top=1.0, hspace=0.05, wspace=0.05)
for i in range(W*H):
    ax = fig.add_subplot(H, W, i + 1, xticks=[], yticks=[])
    ax.imshow(x_train[i].reshape((28, 28)), cmap='gray')
plt.show()

# 学習させやすいようにデータを整形する（次元数を 28x28 の二次元から 784 の一次元に削減し、各値を 0.0 - 1.0 の間に収まるように調整）
# （配列のひとつひとつの要素が、画像の画素 1つに相当する）
x_train = x_train.reshape(-1, 784)/255.0
x_test = x_test.reshape(-1, 784)/255.0
# 正解データは One-Hot ベクトルの形式に変換
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# 読み込んだデータの、次元ごとの要素数を表示
print("x_train（学習用の説明変数データ）の要素数（調整後）: ")
print(x_train.shape)
print("y_train（学習用の目的変数データ） の要素数（調整後）: ")
print(y_train.shape)
print("x_test（検証用の説明変数データ） の要素数（調整後）: ")
print(x_test.shape)
print("y_test（検証用の目的変数データ） の要素数（調整後）: ")
print(y_test.shape)
print("x_train のデータ 1件分のサンプル（調整後）: ")
print(x_train[0])
print("y_train のデータ 10件分のサンプル（調整後）: ")
print(y_train[:10])

# ランダムフォレストで分類問題を解く

In [ ]:
# RandomForestClassifier を用いてランダムフォレストモデルを実装

# ハイパーパラメータ
n_estimators = 10 # 決定木の数。多くすると精度が上がるかもしれない。
max_depth = 8 # 木の枝分かれの深さの最大値。大きくすると精度が上がる傾向が見られるが、過学習の恐れもあり。
criterion = 'gini' # 評価指標。"gini": ジニ係数。"entropy": エントロピー
min_samples_leaf = 8 # 枝を分割するために必要な最小データサイズ。小さくすると精度が上がる傾向が見られるが、過学習の恐れもあり。
min_samples_split = 4 # ひとつの枝の末端（＝葉）に、最低限格納されていなければならないデータ数。。小さくすると精度が上がる傾向が見られるが、過学習の恐れも。

classifier = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, criterion=criterion,
                                    min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split, random_state=1234)
classifier.fit(x_train, y_train)

# Accuracy（正答率）を表示（学習用データの場合）
print('Accuracy（x_train） = {:.3f}%'.format(100 * classifier.score(x_train, y_train)))
# Accuracy（正答率）を表示（検証用データの場合）
print('Accuracy（x_test） = {:.3f}%'.format(100 * classifier.score(x_test, y_test)))

# 予測結果
y_test_predicted = classifier.predict(x_test)

# 検証用データの画像と、その予測結果を並べて表示
print("x_test の予測結果 10件分（ One-Hot ベクトル表現を元に戻して表示）: ")
for i in range(10):
    print(argmax(y_test_predicted[i]))
print("x_test の画像データ 10件分: ")
fig = plt.figure(figsize=(10, 4))
fig.subplots_adjust(left=0, right=1, bottom=0, top=1.0, hspace=0.05, wspace=0.05)
for i in range(10):
    ax = fig.add_subplot(H, W, i + 1, xticks=[], yticks=[])
    ax.imshow(x_test[i].reshape((28, 28)), cmap='gray')
plt.show()

# ランダムフォレストモデルの考察

- ランダムフォレストモデルのハイパーパラメータ（ n_estimators、max_depth、criterion、min_samples_split、min_samples_leaf ）の値を変更すると、Accuracy がどのように変化するでしょうか。

- 検証用データ（ x_test ）のほうの Accuracy が最大となるように、これらのハイパーパラメータを調整してみましょう。

# MLP（Multi-Layer Perceptron、多層パーセプトロン）で分類問題を解く

In [ ]:
# MLP モデルを構築

model = Sequential()

# 　入力層（ 784 -> 256 ）
model.add(Dense(units=256, input_shape=(784,)))
# 入力層の活性化関数として relu 関数を指定（ sigmoid 関数なども指定できるが、誤差逆伝播の際に勾配消失の可能性が出てくる）
model.add(Activation('relu'))

# 隠れ層（ 256 -> 100。基本的には各層のニューロン数を小さくする方向でモデルを構築する ）
model.add(Dense(units=100))
# 隠れ層の活性化関数として relu 関数を指定
model.add(Activation('relu'))

# （たとえば、以下のコメントを外して、2つ目の隠れ層（ 100 -> 50 ）を加えると、予測精度はどう変化するだろうか）
#model.add(Dense(units=50))
#model.add(Activation('relu'))

# 出力層（ 100 -> 10 ）
# 出力層は、分類したい種類（クラス）の個数と一致させる（今回の場合は 10）
model.add(Dense(units=10))
# 　出力層の活性化関数として softmax 関数を指定
# softmax 関数を通すことで、出力層の総和が 1 になるため、それぞれの分類に属する確率を表すことができる
model.add(Activation('softmax'))

# ハイパーパラメータ：
# loss：損失関数。分類問題であるため、交差エントロピー誤差を表す categorical_crossentropy を指定
# optimizer：最適化アルゴリズム。（参考： https://keras.io/ja/optimizers/ ）
optimizer = 'sgd' # 'rmsprop'、'adagrad', 'adadelta', 'adam' など。後述の「参考：Optimizer（最適化アルゴリズム）」も参照
# metrics：評価関数。今回は正解率を表す accuracy を指定
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# モデルのネットワーク構造を可視化
plot_model(model, show_shapes=True, to_file='model.png')

## 参考：Optimizer（最適化アルゴリズム）

Optimizer の最適化イメージ

- 参考： https://postd.cc/optimizing-gradient-descent/

- Adadelta、Rmsprop などの Optimizer が、すぐに最適解に到達できているのに対して、SGD などはなかなか局所解から抜け出せていないように見える


![Optimizer の最適化イメージ](http://sebastianruder.com/content/images/2016/01/saddle_point_evaluation_optimizers.gif)

In [ ]:
# 構築した MLP モデルを用いて学習

# batch_size： ここで指定したデータ件数毎に勾配を更新（小さくするとより細かい頻度で勾配の更新が反映されるようになるが、学習に時間がかかるようになる）
batch_size = 1000
# epochs： 学習の反復回数
epochs = 10
# verbose： 学習の進捗状況の表示レベル
# validation_split： 検証用として取り分けるデータの割合

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0.1)

In [ ]:
# 性能評価
score_train = model.evaluate(x_train, y_train, verbose=0)
score_test = model.evaluate(x_test, y_test, verbose=0)
# Accuracy（正答率）を表示（学習用データの場合）
print('Accuracy（x_train） = {:.3f}%'.format(100 * score_train[1]))
# Accuracy（正答率）を表示（検証用データの場合）
print('Accuracy（x_test） = {:.3f}%'.format(100 * score_test[1]))

# MLP モデルの考察

- ランダムフォレストモデルと比較して、Accuracy（正答率）にどのような違いが見られたでしょうか。

- MLP モデルの隠れ層を追加すると、学習結果がどのように変化するでしょうか。また、各層の unit 数（入力の 784、出力の 10 以外の、途中段階の unit 数）を変更すると、どうでしょうか？

- Optimizer（最適化アルゴリズム）を変更すると、学習結果がどのように変化するでしょうか。

- 検証用データ（ x_test ）のほうの Accuracy が最大となるように、これらを調整してみましょう。